In [4]:
import re, os
import unicodedata
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import nltk.sentiment

from requests import get
from bs4 import BeautifulSoup

from wordcloud import WordCloud


plt.rc('figure', figsize=(13, 7))
plt.style.use('seaborn-darkgrid')

In [5]:
import acquire

---

## Acquire data

In [6]:
def get_all_repository_urls(url): # repository url
    # Get max page.
    response = get(url)
    soup = BeautifulSoup(response.text)
    print('Finding max page for repositories...')
    max_page = int(soup.find('div', role='navigation').text[-6])
    print(f'Max page found: {max_page}')
    page = 1
    repository_links = []
    print('Starting loop...')
    for n in range(max_page):
        print(n+1, 'iteration')
        print(f'Pulling data from {url}')
        # Reset soup.
        response = get(url)
        soup = BeautifulSoup(response.text)
        # Get all the repositories from the page.
        repositories = soup.find_all('a', itemprop='name codeRepository') 
        print('Fetching links for repositories...')
        for repo in repositories:
            repository_links.append(repo.get('href'))
        git = 'https://github.com/'
        next_page = soup.find('a', class_='next_page').get('href')[:-1]
        ## Use this line of code to get the url for the next page.
        if page <= 4:
            url = git + next_page + str(page + 1)
            page += 1
        else:
            return repository_links


In [7]:
repository_links = get_all_repository_urls('https://github.com/orgs/apple/repositories')

Finding max page for repositories...
Max page found: 6
Starting loop...
1 iteration
Pulling data from https://github.com/orgs/apple/repositories
Fetching links for repositories...
2 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=2
Fetching links for repositories...
3 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=3
Fetching links for repositories...
4 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=4
Fetching links for repositories...
5 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=5
Fetching links for repositories...


In [8]:
repository_links

['/apple/llvm-project',
 '/apple/swift',
 '/apple/sourcekit-lsp',
 '/apple/foundationdb',
 '/apple/swift-docc',
 '/apple/swift-protobuf',
 '/apple/swift-llbuild',
 '/apple/swift-syntax',
 '/apple/swift-package-manager',
 '/apple/swift-source-compat-suite',
 '/apple/swift-markdown',
 '/apple/swift-docc-symbolkit',
 '/apple/swift-docc-render-artifact',
 '/apple/swift-lmdb',
 '/apple/swift-tools-support-core',
 '/apple/swift-driver',
 '/apple/indexstore-db',
 '/apple/swift-stress-tester',
 '/apple/swift-xcode-playground-support',
 '/apple/swift-integration-tests',
 '/apple/swift-cmark',
 '/apple/swift-corelibs-libdispatch',
 '/apple/swift-corelibs-foundation',
 '/apple/swift-corelibs-xctest',
 '/apple/swift-experimental-string-processing',
 '/apple/swift-docker',
 '/apple/swift-installer-scripts',
 '/apple/swift-llbuild2',
 '/apple/swift-docc-render',
 '/apple/swift-evolution',
 '/apple/servicetalk',
 '/apple/swift-nio-ssh',
 '/apple/password-manager-resources',
 '/apple/swift-nio',
 '/ap

In [9]:
import acquire_jg

In [14]:
df = pd.read_json('data.json')

In [15]:
df

,repo,language,readme_contents
0,apple/llvm-project,None,# Apple's fork of llvm-project\n\nThis is Appl...
1,apple/swift-argument-parser,Swift,# Swift Argument Parser\n\n## Usage\n\nBegin b...
2,apple/swift-docc,Swift,# Swift-DocC\n\nSwift-DocC is a documentation ...
3,apple/swift,C++,"<img src=""https://swift.org/assets/images/swif..."
4,apple/sourcekit-lsp,Swift,# SourceKit-LSP\n\nSourceKit-LSP is an impleme...
...,...,...,...
146,apple/ccs-caldavclientlibrary,HTML,README for CalDAVClientLibrary\n\nINTRODUCTION...
147,apple/ccs-pyosxframeworks,Python,Getting Started\n===============\n\nThis is a ...
148,apple/ccs-pysecuretransport,Python,Getting Started\n===============\n\nOS X Secur...
149,apple/swift-protobuf-test-conformance,None,"<img src=""https://swift.org/assets/images/swif..."


In [16]:
df.language.value_counts()

Swift               65
Python              38
C++                 15
C                   11
JavaScript           2
Java                 2
Jupyter Notebook     2
HTML                 2
Dockerfile           1
Markdown             1
Shell                1
R                    1
Starlark             1
LLVM                 1
Name: language, dtype: int64

Looks like our categories for classification will be Swift, Python, C (combined C & C++), and other. Can make a new column mapping that target.

---

## Prepare Data

- Add column for target language class
- Add column for clean, lemmatized, etc.
- Split word list by language

---

## Explore